In [1]:
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x5c2224d0bd40
OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x5c2224f63b60


In [2]:
%jsroot on

In [3]:
ROOT.EnableImplicitMT(4)
ROOT.TH1.SetDefaultSumw2()

In [4]:
ROOT.ildStyle.SetOptStat(0)
ROOT.ildStyle.SetPalette(ROOT.kBird)
ROOT.ildStyle.SetNumberContours(256)

In [5]:
ROOT.gInterpreter.Declare("#include \"WWTools.h\"")
# ROOT.gInterpreter.Declare("#include <podio/GenericParameters.h>")

True

In [6]:
%%cpp
using namespace ROOT::VecOps;

In [7]:
# tree = ROOT.TTree()
file = ROOT.TFile("data/paris/nano/all.root")
tree = file["events"]
# tree.AddFriend("events", "data/paris/nano/all.root")
tree.AddFriend("events", "data/paris/nano/all_params_flat.root")

df = ROOT.RDataFrame(tree)

n_bins = 100
x_angle = 0.014 # rad

# TODO implement
beamPol1 = 0.
beamPol2 = 0.
lumi = 1000. # inverse fb

In [8]:
df.Describe()

Dataframe from TTree events in file data/paris/nano/all.root
with friend
  events data/paris/nano/all_params_flat.root

Property                Value
--------                -----
Columns in total           93
Columns from defines        0
Event loops run             0
Processing slots            4

Column                                          Type                                                            Origin
------                                          ----                                                            ------
e_charge                                        Float_t                                                         Dataset
e_lvec                                          ROOT::Math::LorentzVector<ROOT::Math::PxPyPzE4D<double> >       Dataset
e_lvec.fCoordinates                             ROOT::Math::PxPyPzE4D<double>                                   Dataset
e_lvec.fCoordinates.fT                          Double_t                                             

In [9]:
# df = df.Vary("nu_lvec", "ROOT::Math::PxPyPzMVector tmp(E_miss_lvec); return RVec<ROOT::Math::PxPyPzMVector>{tmp};", ["recoable_nu"], "nu_variation")
# df = df.Vary(("nu_lvec", "l_lvec", "hadronic_lvec"), """
#               ROOT::Math::PxPyPzMVector e_miss(E_miss_lvec);
#               auto boost = collision_lvec.BoostToCM();
#               return RVec<RVec<ROOT::Math::PxPyPzMVector>>{
#                 {nu_lvec, e_miss, e_miss, ROOT::Math::VectorUtil::boost(nu_lvec, boost), ROOT::Math::VectorUtil::boost(e_miss, boost)},
#                 {l_lvec + FSR_lvec, l_lvec, l_lvec + FSR_lvec, ROOT::Math::VectorUtil::boost(l_lvec, boost), ROOT::Math::VectorUtil::boost(l_lvec + FSR_lvec, boost)},
#                 {hadronic_lvec, hadronic_lvec, hadronic_lvec, ROOT::Math::VectorUtil::boost(hadronic_lvec, boost), ROOT::Math::VectorUtil::boost(hadronic_lvec, boost)}
#               };
# """, ["FSR", "emiss", "FSR_emiss", "boosted", "boosted_FSR_emiss"], "initial_lvecs")
# # FIXME needs to be commented out when not using rescanned file
# df = df.Vary("weight", "weight * alt_weight", ["g1z_up"], "aTGC")

In [10]:
# do weight calculations: for now only unpolarized
# should happen before any cuts
total_events = tree.GetEntries()

df = df.Define("pol_weight", "1./4.")
# df = df.Define("lumi_weight", "1")
df = df.Define("lumi_weight", f"{lumi} * params_crossSection / {total_events}.")
df = df.Define("weight", "pol_weight * lumi_weight")

In [11]:
# unboost crossing angle (as a variation?)
# reconstruct neutrino: missing E, w/o ovl, w/ overlay
# ISR? crossing angle, sqrt(s)
# build MC and reconstructed W, reconstructed W will vary

# df = df.Define("sqrt_s_E", "Parameters.get<float>(\"Energy\").value()")
df = df.Define("sqrt_s_E", "params_Energy")
# XXX: ignoring the electron mass here
df = df.Define("sqrt_s_lvec", f"ROOT::Math::PxPyPzEVector(sqrt_s_E * sin({x_angle}/2.), 0, 0, sqrt_s_E)")

# TODO correct electron mass back to m_e, maybe as a variation
# df = df.Redefine("e_lvec", "vec = PxPyPzEVector(e_lvec), vec.SetE")

# FIXME debug
df = df.Define("visible_lvec", "e_lvec + isr1_lvec + isr2_lvec + ovl_lvec + had_sum_lvec")
# TODO: add 3 variations: no removal, naive removal (Andre), cheated removal
# Andre's cut: log_10 Pt > 0.2 sin theta + something
# needs to be supplied from stage1!
df = df.Define("identified_ovl_lvec", "ROOT::Math::PxPyPzEVector()")
# TODO variations: none, fitted/reconstructed, cheated ISR goes here
df = df.Define("identified_isr_lvec", "ROOT::Math::PxPyPzEVector()")


In [12]:
df = df.Vary("identified_ovl_lvec", "RVec<ROOT::Math::PxPyPzEVector>({ovl_lvec})", ["cheated"], "ovl")
df = df.Vary("identified_isr_lvec", "RVec<ROOT::Math::PxPyPzEVector>({isr1_lvec + isr2_lvec})", ["cheated"], "isr")

In [13]:

df = df.Define("visible_corrected_lvec", "visible_lvec - identified_ovl_lvec - identified_isr_lvec")
df = df.Define("missing_E_lvec", "sqrt_s_lvec - visible_corrected_lvec")
df = df.Define("reco_nu_lvec", "missing_E_lvec")
# TODO: correct nu mass back to 0?

df = df.Define("W_lep_lvec", "e_lvec + reco_nu_lvec")
df = df.Define("W_had_lvec", "visible_corrected_lvec - e_lvec")

In [14]:
df = df.Define("W_lep_invM", "W_lep_lvec.mass()")
df = df.Define("W_had_invM", "W_had_lvec.mass()")

In [15]:

df = df.Define("e_minus_vec", f"ROOT::Math::XYZVector(sin({x_angle}/2.), 0, cos({x_angle}/2.))")

df = df.Define("CosTheta_W_lep", "ROOT::Math::VectorUtil::CosTheta(W_lep_lvec, e_minus_vec)")
df = df.Define("CosTheta_W_had", "ROOT::Math::VectorUtil::CosTheta(W_had_lvec, e_minus_vec)")

df = df.Define("l_star_lvec", "WWTools::starVector(W_lep_lvec, e_lvec)")

df = df.Define("l_theta_star", "l_star_lvec.Theta()")
df = df.Define("l_phi_star", "l_star_lvec.Phi()")
df = df.Define("l_cosTheta_star", "cos(l_theta_star)")

In [16]:
# Filters go here:
# df = df.Filter("events.params_WWCategorisation_TrueCat == 2")
# df = df.Filter("events.params_WWCategorisation_RecoCat == 2")
df = df.Filter("params_WWCategorisation_RecoCatAdvanced == 2")
# df = df.Filter("abs(W_lep_invM - 80) < 15")

In [17]:
h_W_lep_invM = df.Histo1D(("", ";W_lep_invM [GeV]", 300, -100., 200.), "W_lep_invM", "weight")
h_W_had_invM = df.Histo1D(("", ";W_had_invM [GeV]", 200, 0., 200.), "W_had_invM", "weight")

v_W_lep_invM = ROOT.RDF.Experimental.VariationsFor(h_W_lep_invM)
v_W_had_invM = ROOT.RDF.Experimental.VariationsFor(h_W_had_invM)

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x5c22295b0cb0
OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x5c22294fb4f0


In [18]:
h_CosTheta_W_minus_had = df.Filter("e_charge > 0").Histo1D(("", ";cos #theta_{W^{-}_{had}}; Events / 0.05", n_bins, -1., 1.), "CosTheta_W_had", "weight")
h_CosTheta_W_minus_lep = df.Filter("e_charge < 0").Histo1D(("", ";cos #theta_{W^{-}_{lep}}; Events / 0.05", n_bins, -1., 1.), "CosTheta_W_lep", "weight")
h_CosTheta_W_plus_had = df.Filter("e_charge < 0").Histo1D(("", ";cos #theta_{W^{+}_{had}}; Events / 0.05", n_bins, -1., 1.), "CosTheta_W_had", "weight")
h_CosTheta_W_plus_lep = df.Filter("e_charge > 0").Histo1D(("", ";cos #theta_{W^{+}_{lep}}; Events / 0.05", n_bins, -1., 1.), "CosTheta_W_lep", "weight")

v_CosTheta_W_minus_had = ROOT.RDF.Experimental.VariationsFor(h_CosTheta_W_minus_had)
v_CosTheta_W_minus_lep = ROOT.RDF.Experimental.VariationsFor(h_CosTheta_W_minus_lep)
v_CosTheta_W_plus_had = ROOT.RDF.Experimental.VariationsFor(h_CosTheta_W_plus_had)
v_CosTheta_W_plus_lep = ROOT.RDF.Experimental.VariationsFor(h_CosTheta_W_plus_lep)

In [19]:
h_l_minus_cosTheta_star = df.Filter("e_charge < 0").Histo1D(("", ";cos #theta*_{e^{-}}", n_bins, -1., 1.), "l_cosTheta_star", "weight")
h_l_plus_cosTheta_star = df.Filter("e_charge > 0").Histo1D(("", ";cos #theta*_{e^{+}}", n_bins, -1., 1.), "l_cosTheta_star", "weight")

v_l_minus_cosTheta_star = ROOT.RDF.Experimental.VariationsFor(h_l_minus_cosTheta_star)
v_l_plus_cosTheta_star = ROOT.RDF.Experimental.VariationsFor(h_l_plus_cosTheta_star)

h_l_minus_phi_star = df.Filter("e_charge < 0").Histo1D(("", ";#phi*_{e-}", n_bins, -3.14159, 3.14159), "l_phi_star", "weight")
h_l_plus_phi_star = df.Filter("e_charge > 0").Histo1D(("", ";#phi*_{e+}", n_bins, -3.14159, 3.14159), "l_phi_star", "weight")

v_l_minus_phi_star = ROOT.RDF.Experimental.VariationsFor(h_l_minus_phi_star)
v_l_plus_phi_star = ROOT.RDF.Experimental.VariationsFor(h_l_plus_phi_star)

In [20]:
df = df.Define("nu_invM", "reco_nu_lvec.M()")
h_nu_invM = df.Histo1D("nu_invM")
v_nu_invM = ROOT.RDF.Experimental.VariationsFor(h_nu_invM)

c_nu_invM = ROOT.TCanvas()
h_nu_invM.Draw()
v_nu_invM["ovl:cheated"].SetMarkerColor(ROOT.kRed)
v_nu_invM["ovl:cheated"].Draw("same")
c_nu_invM.Draw()

In [21]:
c_W_lep_invM = ROOT.TCanvas()
v_W_lep_invM["ovl:cheated"].SetMarkerColor(ROOT.kRed)
v_W_lep_invM["ovl:cheated"].Draw()
h_W_lep_invM.Draw("same")
# v_W_lep_invM["isr:cheated"].SetMarkerColor(ROOT.kBlue)
# v_W_lep_invM["isr:cheated"].Draw("same")
c_W_lep_invM.Draw()

c_W_had_invM = ROOT.TCanvas()
v_W_had_invM["ovl:cheated"].SetMarkerColor(ROOT.kRed)
v_W_had_invM["ovl:cheated"].Draw()
h_W_had_invM.Draw("same")
c_W_had_invM.Draw()

In [22]:
c_CosTheta_W_plus_had = ROOT.TCanvas()
h_CosTheta_W_plus_had.Draw()
v_CosTheta_W_plus_had["ovl:cheated"].SetMarkerColor(ROOT.kRed)
v_CosTheta_W_plus_had["ovl:cheated"].Draw("same")
c_CosTheta_W_plus_had.Draw()

c_CosTheta_W_minus_had = ROOT.TCanvas()
h_CosTheta_W_minus_had.Draw()
v_CosTheta_W_minus_had["ovl:cheated"].SetMarkerColor(ROOT.kRed)
v_CosTheta_W_minus_had["ovl:cheated"].Draw("same")
c_CosTheta_W_minus_had.Draw()

c_CosTheta_W_plus_lep = ROOT.TCanvas()
h_CosTheta_W_plus_lep.Draw()
v_CosTheta_W_plus_lep["ovl:cheated"].SetMarkerColor(ROOT.kRed)
v_CosTheta_W_plus_lep["ovl:cheated"].Draw("same")
c_CosTheta_W_plus_lep.Draw()

c_CosTheta_W_minus_lep = ROOT.TCanvas()
h_CosTheta_W_minus_lep.Draw()
v_CosTheta_W_minus_lep["ovl:cheated"].SetMarkerColor(ROOT.kRed)
v_CosTheta_W_minus_lep["ovl:cheated"].Draw("same")
c_CosTheta_W_minus_lep.Draw()

In [23]:
c_l_minus_cosTheta_star = ROOT.TCanvas()
v_l_minus_cosTheta_star["ovl:cheated"].SetMarkerColor(ROOT.kRed)
v_l_minus_cosTheta_star["ovl:cheated"].SetMinimum(0)
v_l_minus_cosTheta_star["ovl:cheated"].Draw()
h_l_minus_cosTheta_star.Draw("same")
c_l_minus_cosTheta_star.Draw()

c_l_plus_cosTheta_star = ROOT.TCanvas()
v_l_plus_cosTheta_star["ovl:cheated"].SetMarkerColor(ROOT.kRed)
v_l_plus_cosTheta_star["ovl:cheated"].SetMinimum(0)
v_l_plus_cosTheta_star["ovl:cheated"].Draw()
h_l_plus_cosTheta_star.Draw("same")
c_l_plus_cosTheta_star.Draw()

c_l_minus_phi_star = ROOT.TCanvas()
v_l_minus_phi_star["ovl:cheated"].SetMarkerColor(ROOT.kRed)
v_l_minus_phi_star["ovl:cheated"].SetMinimum(0)
v_l_minus_phi_star["ovl:cheated"].SetMaximum(50000)
v_l_minus_phi_star["ovl:cheated"].Draw()
h_l_minus_phi_star.Draw("same")
c_l_minus_phi_star.Draw()

c_l_plus_phi_star = ROOT.TCanvas()
v_l_plus_phi_star["ovl:cheated"].SetMarkerColor(ROOT.kRed)
v_l_plus_phi_star["ovl:cheated"].SetMinimum(0)
v_l_plus_phi_star["ovl:cheated"].SetMaximum(50000)
v_l_plus_phi_star["ovl:cheated"].Draw()
h_l_plus_phi_star.Draw("same")
c_l_plus_phi_star.Draw()